# Explore here

In [ ]:
# Your code here

import pandas as pd

# Add data to variable 'total data'
total_data = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv', delimiter=';')

# Insert raw data inside raw folder inside root.
total_data.to_csv('/workspaces/ginotomasd-ml-project-logisticalregression/data/raw/total_data.csv')

# Check for columns and rows
total_data.shape

# Check values inside
total_data.info()

# We can see that only 1 column contains all the rows, and we fixed that by adding delimiters to our 'total_data' variable.
    # Now each variable is stored in an individual column, and we are able to check for nulls
        # We do not find any nulls, all columns contain the same number of rows.
            # We discard the possibility of duplicates

duplicates = total_data.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

total_data[duplicates]

# After checking duplicates, we discover they are not duplicates, as they contain different IDs but arent shown because of the way the data is structured. There is no ID column.








<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 